<a href="https://colab.research.google.com/github/weso500/QMLPublicationRuns/blob/main/HardFault3TEPFMT_Pauli_Rep1_Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00


In [2]:
pip install qiskit-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
  Attempting uninstall: qiskit
    Found existing installation: qiskit 2.2.3
    Uninstalling qiskit-2.2.3:
      Successfully uninstalled qiskit-2.2.3


In [3]:
pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 6.0 MB/s eta 0:00:00


In [4]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [5]:
df_FaultFree = pyreadr.read_r(r'/content/drive/MyDrive/FDA Journal/TEP_FaultFree_Training.RData')['fault_free_training']
df_Faulty = pyreadr.read_r(r'/content/drive/MyDrive/FDA Journal/TEP_Faulty_Training.RData')['faulty_training']

In [6]:
DF = pd.concat([df_FaultFree,df_Faulty]) # Join both fault free and faulty dataset
DF.head()

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461


In [7]:
columns_to_keep = [
    'faultNumber',
    'simulationRun',
    'sample',

    # Core thermal indicators
    'xmeas_9',   # Reactor temperature (directly affected by cooling fault)
    'xmeas_21',  # Reactor cooling water outlet temp (strong fault signature)
    'xmv_10',    # Reactor cooling water flow valve (control response)

    # Secondary thermal propagation effects
    'xmeas_18',  # Stripper temperature
    'xmeas_11',  # Separator temperature
    'xmeas_22',  # Condenser cooling water outlet temperature

    # Compositional response (reaction chemistry changes)
    'xmeas_41',  # Composition of H product
    'xmeas_33',  # Composition of E purge
    'xmeas_35',  # Composition of G purge

    # Reactor state
    'xmeas_8',   # Reactor level (slight coupling with heat exchange)

    # Flow dynamics and purge control
    'xmeas_10',  # Purge rate (minor but diagnostic for late-stage imbalance)
]

DF = DF[columns_to_keep]
display(DF.head())

,faultNumber,simulationRun,sample,xmeas_9,xmeas_21,xmv_10,xmeas_18,xmeas_11,xmeas_22,xmeas_41,xmeas_33,xmeas_35,xmeas_8,xmeas_10
0,0.0,1.0,1,120.41,94.640,41.258,65.732,80.044,77.047,43.828,18.579,4.8436,74.863,0.33818
1,0.0,1.0,2,120.41,94.595,41.359,65.716,80.078,77.434,43.828,18.579,4.8436,75.000,0.33620
2,0.0,1.0,3,120.42,94.605,41.199,65.732,80.220,77.466,43.828,18.765,4.8543,74.771,0.33563
3,0.0,1.0,4,120.39,94.473,41.643,65.781,80.305,77.443,43.828,18.765,4.8543,75.224,0.33553
4,0.0,1.0,5,120.39,94.678,41.507,65.788,80.064,76.947,43.828,18.666,4.8304,75.388,0.32632


In [8]:

from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import PauliFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

dimension = 11
feature_map = PauliFeatureMap(feature_dimension=dimension, reps=1, entanglement="full")

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.kernel_approximation import Nystroem
from joblib import Parallel, delayed

# ---- Helper: 95% CI for a 1D array-like
def mean_ci(arr, alpha=0.05):
    arr = np.asarray(arr, dtype=float)
    n = len(arr)
    m = np.mean(arr)
    s = np.std(arr, ddof=1) if n > 1 else 0.0
    z = 1.96  # normal approximation
    half = z * (s / np.sqrt(max(n, 1)))
    return m, m - half, m + half

def youden_threshold(y_true, scores):
    """Return the threshold that maximizes Youden's J = TPR - FPR."""
    fpr, tpr, thr = metrics.roc_curve(y_true, scores, pos_label=1)
    # Avoid non-finite thresholds (first element can be inf)
    mask = np.isfinite(thr)
    fpr, tpr, thr = fpr[mask], tpr[mask], thr[mask]
    j = tpr - fpr
    if len(j) == 0:
        # fallback: median score if something degenerate happens
        return np.median(scores)
    idx = np.argmax(j)
    return thr[idx]

# === Define helper for one trial ===
def run_trial(i, fault_id=1, n_train_normals=100, n_test_normals=70, n_test_faulty=5, nystrom_components=64, nu=0.05):
    rng = np.random.RandomState(i)
    sc = StandardScaler()

    # ----- Build train/test splits
    fault_free_all = DF[DF['faultNumber'] == 3].iloc[::50, 3:]
    TEPtrain_df = fault_free_all.sample(n=n_train_normals, random_state=i)
    TEPtrain = sc.fit_transform(TEPtrain_df)

    # Ensure test normals don't overlap the selected train rows
    remaining_normals = fault_free_all.drop(TEPtrain_df.index)
    TEP_test_normals_df = remaining_normals.sample(n=n_test_normals, random_state=i+100)

    faulty_all = DF[DF['faultNumber'] == fault_id].iloc[::30, 3:]
    faulty_samples_df = faulty_all.sample(n=n_test_faulty, random_state=i+200)

    # Align columns and stack
    test_df = pd.concat(
        [pd.DataFrame(TEP_test_normals_df, columns=faulty_samples_df.columns), faulty_samples_df],
        axis=0,
        ignore_index=True
    )
    TEPtest = sc.transform(test_df)

    y_true = np.concatenate((
        np.zeros(n_test_normals, dtype=int),
        np.ones(n_test_faulty, dtype=int)
    ))

    # --- Nyström approximation of the quantum kernel ---
    n_components = min(nystrom_components, len(TEPtrain) - 1)
    Phi = Nystroem(
        kernel=lambda A, B=None: kernel.evaluate(x_vec=A, y_vec=B),
        n_components=n_components,
        random_state=i
    )
    Ztr = Phi.fit_transform(TEPtrain)
    Zte = Phi.transform(TEPtest)

    # Precomputed Gram matrices
    Ktr = Ztr @ Ztr.T
    Kte = Zte @ Ztr.T

    # One-Class SVM (nu ~ anomaly prior)
    ocsvm = OneClassSVM(kernel='precomputed', nu=nu)
    ocsvm.fit(Ktr)

    # Scores: higher = more anomalous
    scores = -ocsvm.decision_function(Kte)

    # --- Threshold-free metrics
    roc_auc = metrics.roc_auc_score(y_true, scores)
    pr_auc  = metrics.average_precision_score(y_true, scores)

    # --- Youden-optimal threshold
    thr = youden_threshold(y_true, scores)
    y_pred = (scores >= thr).astype(int)  # 1 = anomaly

    precision = metrics.precision_score(y_true, y_pred, zero_division=0)
    recall    = metrics.recall_score(y_true, y_pred, zero_division=0)
    f1        = metrics.f1_score(y_true, y_pred, zero_division=0)
    accuracy  = metrics.accuracy_score(y_true, y_pred)

    return {
        "roc_auc": roc_auc,
        "pr_auc": pr_auc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy,
        "threshold": thr
    }

# === Run in parallel ===
N_RUNS = 30
results = Parallel(n_jobs=8)(
    delayed(run_trial)(i, fault_id=1, n_train_normals=100, n_test_normals=70, n_test_faulty=5,
                       nystrom_components=64, nu=0.05)
    for i in range(N_RUNS)
)
df = pd.DataFrame(results)

# === Summaries with 95% CI ===
summary_rows = []
for metric in ["roc_auc", "pr_auc", "precision", "recall", "f1", "accuracy", "threshold"]:
    m, lo, hi = mean_ci(df[metric].values)
    summary_rows.append({"metric": metric, "mean": m, "ci95_lo": lo, "ci95_hi": hi})

summary = pd.DataFrame(summary_rows).set_index("metric")

print("\n=== Cross-run metrics (mean ± 95% CI) using Youden-optimal threshold ===")
for metric, row in summary.iterrows():
    if metric == "threshold":
        print(f"{metric:>9}: {row['mean']:.4f}  (95% CI: {row['ci95_lo']:.4f} .. {row['ci95_hi']:.4f})")
    else:
        print(f"{metric:>9}: {row['mean']:.3f}  (95% CI: {row['ci95_lo']:.3f} .. {row['ci95_hi']:.3f})")






=== Cross-run metrics (mean ± 95% CI) using Youden-optimal threshold ===
  roc_auc: 0.832  (95% CI: 0.800 .. 0.864)
   pr_auc: 0.405  (95% CI: 0.346 .. 0.465)
precision: 0.273  (95% CI: 0.219 .. 0.326)
   recall: 0.873  (95% CI: 0.829 .. 0.917)
       f1: 0.393  (95% CI: 0.340 .. 0.446)
 accuracy: 0.782  (95% CI: 0.742 .. 0.823)
threshold: 0.0001  (95% CI: 0.0000 .. 0.0001)
